In [4]:
import pandas as pd
from dateutil import relativedelta
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
import pymmwr
import random
from scipy import stats


def dominant_subtype(year):
    if year in ['2009', '2009Pan', '2010', '2014', '2016']:
        subtypes = ['H1N1', 'H1N1pdm']
    else:
        subtypes = ['H3N2']
    return subtypes

def season_to_season_float(season_str):
    if season_str == '2009Pan':
        s = 2009.5
    else:
        s = float(season_str)
    return s

def season_float_to_season(season_float):
    if season_float == 2009.5:
        s = '2009Pan'
    else:
        s = str(int(season_float))
    return s
    
df_by = pd.read_csv('..//raw_data/cases_of_dominant_subtype_by_birth_year.csv')
df_by = df_by.groupby(['Season', 'Birth_year', 'Vaccination_status', 'Season_section']).sum()
df_age = pd.read_csv('../raw_data/cases_of_dominant_subtype_by_age.csv')
df_age = df_age.groupby(['Season', 'Age', 'Vaccination_status', 'Season_section']).sum()

total_cases = pd.read_csv('../data/extrapolated_case_totals.csv')
cases_at_first_week = pd.read_csv('../raw_data/cases_at_first_two_weeks.csv')
cases_at_first_week['age'] = cases_at_first_week.Season.apply(np.floor) - 1 - cases_at_first_week.Birth_year
cases_at_last_week = pd.read_csv('../raw_data/cases_at_last_two_weeks.csv')
cases_at_last_week['age'] = cases_at_last_week.Season.apply(np.floor) - 1 - cases_at_last_week.Birth_year

In [5]:
for t in range(0, 1000):

    np.random.seed(t)
    bycopy = df_by.copy()
    agecopy = df_age.copy()
    total_new_cases = 0
    for season, seasondf in total_cases.groupby('Season'):
        season_float = season_to_season_float(season)
        new_rows = []
        temp = seasondf.copy()
        temp.index = temp.Period

        try:
            start_cases = temp.loc['start', 'Cases']
        except KeyError:
            start_cases = 0
            
        try:
            end_cases = temp.loc['end', 'Cases']
        except KeyError:
            end_cases = 0

        if start_cases > 0:
            total_new_cases += start_cases
            p = cases_at_first_week[cases_at_first_week.Season == season_float].case_probability
            c = np.random.multinomial(start_cases, p)
            new_start = cases_at_first_week[cases_at_first_week.Season == season_float].copy()
            new_start['Cases'] = c
            
            new_start = new_start[new_start.Cases > 0]
            
            for index, row in new_start.iterrows():
                age_key = (season, row.age, row['Vaccination status'], 'Start')
                by_key = (season, row.Birth_year, row['Vaccination status'], 'Start')
                try:
                    agecopy.loc[age_key, 'Count'] += row.Cases
                except KeyError:
                    agecopy.loc[age_key, 'Count'] = row.Cases
                try:
                    bycopy.loc[by_key, 'Count'] += row.Cases
                except KeyError:
                    bycopy.loc[by_key, 'Count'] = row.Cases
 

        if end_cases > 0:
            total_new_cases += end_cases
            p = cases_at_last_week[cases_at_last_week.Season == season_float].case_probability
            c = np.random.multinomial(end_cases, p)
            new_end = cases_at_last_week[cases_at_last_week.Season == season_float].copy()
            new_end['Cases'] = c
            new_start = new_start[new_start.Cases > 0]
            for index, row in new_end.iterrows():
                age_key = (season, row.age, row['Vaccination status'], 'End')
                by_key = (season, row.Birth_year, row['Vaccination status'], 'End')
                
                try:
                    agecopy.loc[age_key, 'Count'] += row.Cases
                except KeyError:
                    agecopy.loc[age_key, 'Count'] = row.Cases
                try:
                    bycopy.loc[by_key, 'Count'] += row.Cases
                except KeyError:
                    bycopy.loc[by_key, 'Count'] = row.Cases
        
    bycopy.to_csv('./simulated_data/simulation_by_birth_year_%s.csv'%t)
    agecopy.to_csv('./simulated_data/simulation_by_age_%s.csv'%t)
    

